[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/security-lab-notebooks/blob/main/template_notebook.ipynb### 1. Install atlas CLI on the notebook environment.

In [ ]:
# CODE_BLOCK_1

!wget https://fastdl.mongodb.org/mongocli/mongodb-atlas-cli_1.31.0_linux_x86_64.deb
!dpkg -i mongodb-atlas-cli_1.31.0_linux_x86_64.deb

## Might require:
## !sudo dpkg -i mongodb-atlas-cli_1.31.0_linux_x86_64.deb

### 2. Authenticate the CLI against your Atlas organization.

In [ ]:
# CODE_BLOCK_2

!atlas auth login

### 3. Create a new project called \"MySecureProj\"

In [ ]:
# CODE_BLOCK_3

import getpass
org_id = getpass.getpass('Enter your org ID: ')
!atlas projects create MySecureProj --orgId {org_id}

### 4. Create your challenge cluster

In [ ]:
# CODE_BLOCK_4

project_id = getpass.getpass('Enter your project ID: ')
!atlas clusters create MyNewCluster --provider AWS --region US_EAST_1 --tier M0 --projectId {project_id}

### 5. Add 'My current IP' temporary into the atlas project

In [ ]:
# CODE_BLOCK_5

from datetime import datetime, timedelta
delete_after = (datetime.utcnow() + timedelta(hours=24)).isoformat() + 'Z'
!atlas accessLists ...

### 6. Check that 'My current IP' was added:

In [ ]:
# CODE_BLOCK_6

!atlas accessLists list --output json --projectId {project_id}

### 7. Let's create a SCRAM user: 'myUser' with User/Password authentication and assign it the 'readWriteAnyDatabase' role.

In [ ]:
# CODE_BLOCK_7

!atlas dbusers create ...

### 8. Lets test our SCRAM user successful creation by performing the authentication process

In [ ]:
# CODE_BLOCK_8

!pip install pymongo dnspython

In [ ]:
# CODE_BLOCK_9

connection = !atlas clusters connectionStrings describe  MyNewCluster --projectId {project_id}
username = 'myUser'
password = 'mySecurePassword'
new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')
print(new_connection)
from pymongo import MongoClient
client = MongoClient(new_connection)
client.list_database_names()

### 10. Let's create  X509 user: 'myX509User' with User/Password authentication and assign it the 'readWriteAnyDatabase' role.

In [ ]:
# CODE_BLOCK_10

!atlas dbusers create ...
!atlas dbusers certs create ...  > /tmp/cert.pem

### 11. Let's test our X509 User

In [ ]:
# CODE_BLOCK_11

username='myX509User'
connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}
new_connection = connection[1].replace('.net', '.net?authSource=%24external&authMechanism=MONGODB-X509')
print(new_connection)
from pymongo import MongoClient
client = MongoClient(new_connection,
                     tlsCertificateKeyFile='/tmp/cert.pem')
client.list_database_names()

### 12. Let's create a user: 'myAdmin' with User/Password authentication and assign it the 'readWriteAnyDatabase' role, but only for the 'MyNewCluster' database.

In [ ]:
# CODE_BLOCK_12

!atlas dbusers create ...

### 13. Let's create a user: 'readOnlyUser' with read-only access to the 'salesDB' database.

In [ ]:
# CODE_BLOCK_13
!atlas customDbRoles create ...
!atlas dbusers create ...

### 14. Let's test that the 'readOnlyUser' cannot insert data into the 'salesDB' database.

In [ ]:
# CODE_BLOCK_14
# Assuming you have the connection string and credentials for the readOnlyUser
username = 'readOnlyUser'
password = 'readOnlyPass'
connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}
new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')
client = MongoClient(new_connection)
db = client['salesDB']
collection = db['mycollection']
try:
  data = {'name': 'John Doe', 'age': 30}
  result = collection.insert_one(data)
  print(f"Inserted document with ID: {result.inserted_id}")
except Exception as e:
  print(f"Error inserting data: {e}")

### 15. Let's test that the 'myAdmin' user can insert data into the 'salesDB' database.

In [ ]:
# CODE_BLOCK_15
# Assuming you have the connection string and credentials for the myAdmin user
username = 'myAdmin'
password = 'secureAdminPass'
connection = !atlas clusters connectionStrings describe MyNewCluster --projectId {project_id}
new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')
client = MongoClient(new_connection)
db = client['salesDB']
collection = db['mycollection']
try:
  data = {'name': 'John Doe', 'age': 30}
  result = collection.insert_one(data)
  print(f"Inserted document with ID: {result.inserted_id}")
except Exception as e:
  print(f"Error inserting data: {e}")

### 16. Install the necessary packages.

In [ ]:
# CODE_BLOCK_16
!pip install pymongo[encryption]
!pip install pymongo
!pip install requests

### 17. Set up the crypt_shared library.

In [ ]:
# CODE_BLOCK_17
import os
from pymongo import MongoClient
from pymongo.encryption import Algorithm, ClientEncryption, QueryType
from pymongo.encryption_options import AutoEncryptionOpts
from bson.codec_options import CodecOptions
from bson import json_util
import json
import requests
import platform
import tempfile
def setup_crypt_shared():
    system = platform.system().lower()
    if system == "linux":
        url = "https://downloads.mongodb.com/linux/mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-6.0.6.tgz"
        filename = "mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-6.0.6.tgz"
    elif system == "darwin":
        url = "https://downloads.mongodb.com/osx/mongo_crypt_shared_v1-macos-x86_64-enterprise-6.0.6.tgz"
        filename = "mongo_crypt_shared_v1-macos-x86_64-enterprise-6.0.6.tgz"
    elif system == "windows":
        url = "https://downloads.mongodb.com/windows/mongo_crypt_shared_v1-windows-x86_64-enterprise-6.0.6.zip"
        filename = "mongo_crypt_shared_v1-windows-x86_64-enterprise-6.0.6.zip"
    else:
        raise OSError("Unsupported operating system")
    response = requests.get(url)
    response.raise_for_status()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".tgz" if system != "windows" else ".zip") as tmp_file:
        tmp_file.write(response.content)
        tmp_file_path = tmp_file.name
    extract_dir = tempfile.mkdtemp()
    if system != "windows":
        os.system(f"tar -xzf {tmp_file_path} -C {extract_dir}")
        lib_path = os.path.join(extract_dir, "lib", "mongo_crypt_v1.so")
    else:
        os.system(f"powershell Expand-Archive -Path {tmp_file_path} -DestinationPath {extract_dir}")
        lib_path = os.path.join(extract_dir, "bin", "mongo_crypt_v1.dll")
    return lib_path
crypt_shared_lib_path = setup_crypt_shared()
print(f"Crypt shared library path: {crypt_shared_lib_path}")

### 18. Connect to your Atlas cluster.

In [ ]:
# CODE_BLOCK_18
connection = !atlas clusters connectionStrings describe  MyNewCluster --projectId {project_id}
username = 'myUser'
password = 'mySecurePassword'
new_connection = connection[1].replace('mongodb+srv://', f'mongodb+srv://{username}:{password}@')
print(new_connection)
from pymongo import MongoClient
client = MongoClient(new_connection)

### 19. Set up encryption key and providers, define the encrypted fields map, create an encrypted client, and set up the key vault.

In [ ]:
# CODE_BLOCK_19

import os
from pymongo import MongoClient
from pymongo.encryption import Algorithm, ClientEncryption, QueryType
from pymongo.encryption_options import AutoEncryptionOpts
from bson.codec_options import CodecOptions
from bson import json_util
import json
import requests
import platform
import tempfile

local_master_key = os.urandom(96)
kms_providers = {"local": {"key": local_master_key}}
key_vault_namespace = "encryption.__keyVault"
kms_provider_name="local"
key_vault_database_name = "encryption"
key_vault_collection_name = "__keyVault"
key_vault_namespace = f"{key_vault_database_name}.{key_vault_collection_name}"
encrypted_database_name = "medicalRecords"
encrypted_collection_name = "patients"

auto_encryption_options = AutoEncryptionOpts(
    kms_providers,
    key_vault_namespace,
    crypt_shared_lib_path=crypt_shared_lib_path
)

encrypted_client = MongoClient(
    new_connection, auto_encryption_opts=auto_encryption_options)

# TODO CODE_BLOCK_19

encrypted_fields_map = ...

client_encryption = ClientEncryption(
    kms_providers=kms_providers,
    key_vault_namespace=key_vault_namespace,
    key_vault_client=encrypted_client,
    codec_options=CodecOptions()
)

# TODO CODE_BLOCK_19

client_encryption.create_encrypted_collection(
    encrypted_client[encrypted_database_name],
    encrypted_collection_name,
    ... ,
    kms_provider_name,
    {},
)


### 20. Insert an encrypted document.

In [ ]:
#  CODE_BLOCK_20

patient_document = {
    "patientName": "Jon Doe",
    "patientId": 12345678,
    "patientRecord": {
        "ssn": "987-65-4320",
        "billing": {
            "type": "Visa",
            "number": "4111111111111111",
        },
    },
}

# TODO CODE_BLOCK_20

encrypted_collection = #  encrypted_client ...
result = encrypted_collection.insert_one(...)
print(f"Inserted document ID: {result.inserted_id}")


### 21. Query the encrypted collection.

In [ ]:
# TODO CODE_BLOCK_21

find_result = encrypted_collection.find_one(...)
print(find_result)

### 22. Query the collection without encryption.

In [ ]:
# CODE_BLOCK_22

reg_mongoclient = MongoClient(new_connection)


# TODO CODE_BLOCK_22

find_result = reg_mongoclient[encrypted_database_name][encrypted_collection_name].find_one(...)
print(find_result)

all_docs = reg_mongoclient[encrypted_database_name][encrypted_collection_name].find()
print("\nAll documents in the collection:")
for doc in all_docs:
    print(doc)